In [1]:
import pandas as pd
from scipy.optimize import curve_fit
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt
# %matplotlib inline
from datetime import datetime, date


C:\Users\etiirine\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df= pd.read_table("Static vacuum test after bakeout.txt",sep='\t')
df.drop(columns=['Full_Range_Turbo','Pirani_1','Injection','XHVTrap','I_Em'], inplace=True)

df.head()

,Time,Penning_1,Penning_2,Live comments
0,28/09/2022 14:54:30,3.400000e-10,4.400000e-10,NaN
1,28/09/2022 14:54:34,3.400000e-10,4.400000e-10,"Static vacuum test, Penning +RGA MID"
2,28/09/2022 14:54:45,3.400000e-10,4.400000e-10,NaN
3,28/09/2022 14:54:55,3.400000e-10,4.400000e-10,Under dynamic vacuum
4,28/09/2022 14:55:05,3.400000e-10,4.400000e-10,NaN


In [3]:
timestamp=[]
for i in df["Time"]:
    timestamp.append(i[11:])
df.insert(1,"Timestamp",timestamp)
#making the column into a datetime object
timestamps=pd.to_datetime(df["Time"],format="%d/%m/%Y %H:%M:%S")


In [4]:
print(timestamps)

0      2022-09-28 14:54:30
1      2022-09-28 14:54:34
2      2022-09-28 14:54:45
3      2022-09-28 14:54:55
4      2022-09-28 14:55:05
               ...        
8450   2022-09-29 09:58:52
8451   2022-09-29 09:58:53
8452   2022-09-29 09:59:04
8453   2022-09-29 09:59:05
8454   2022-09-29 09:59:07
Name: Time, Length: 8455, dtype: datetime64[ns]


In [5]:
#runtime=[]
#for i in range(len(df["Timestamp"])):
#    runtime.append((timestamps[i]-timestamps[0]).dt.total_seconds())
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
df.insert(2,"Elapsed time",runtime)


In [6]:
df.head()

,Time,Timestamp,Elapsed time,Penning_1,Penning_2,Live comments
0,28/09/2022 14:54:30,14:54:30,0.0,3.400000e-10,4.400000e-10,NaN
1,28/09/2022 14:54:34,14:54:34,4.0,3.400000e-10,4.400000e-10,"Static vacuum test, Penning +RGA MID"
2,28/09/2022 14:54:45,14:54:45,15.0,3.400000e-10,4.400000e-10,NaN
3,28/09/2022 14:54:55,14:54:55,25.0,3.400000e-10,4.400000e-10,Under dynamic vacuum
4,28/09/2022 14:55:05,14:55:05,35.0,3.400000e-10,4.400000e-10,NaN


In [9]:
#overall pressure evolution
plt.figure()
plt.plot((df["Elapsed time"]/3600),df['Penning_1'],marker=".", markersize=5,label='PE1')
plt.plot((df["Elapsed time"]/3600),df['Penning_2'],marker=".", markersize=5,label='PE2')
plt.xlabel('Elapsed time (h)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("RT setup pressure evolution")
plt.yscale('log')

In [10]:
#sorting out values which are in the static vacuum region
#use only PE2, due to being more representative of the experiment area in the chamber
values=[]
statvac=df["Penning_2"][0:np.argmax(df["Penning_2"])]
timeperiod=df["Elapsed time"][0:np.argmax(df["Penning_2"])]
print(statvac)
                                        

0      4.400000e-10
1      4.400000e-10
2      4.400000e-10
3      4.400000e-10
4      4.400000e-10
           ...     
546    1.300000e-06
547    1.300000e-06
548    1.400000e-06
549    1.500000e-06
550    1.600000e-06
Name: Penning_2, Length: 551, dtype: float64


In [11]:
#plotting the pressure evolution of the static vacuum 
plt.figure()
plt.plot((timeperiod/60),statvac,marker=".", markersize=5,label='PE2')
plt.xlabel('Elapsed time (min)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title('RT setup pressure evolution under static vacuum')
plt.yscale('log')